In [3]:
from ultralytics import YOLO
import cv2
import csv
import numpy as np
from sort.sort import *

# Your custom utility functions
from util import get_car, read_license_plate, write_csv

# Initialize variables
results = {}
mot_tracker = Sort()
coco_model = YOLO('yolov8m.pt')
license_plate_detector = YOLO('/Users/malik/Desktop/Malik/University/General - GP 1  2023 A/G/automatic-number-plate-recognition-python-yolov8-main-2/best.pt')
cap = cv2.VideoCapture('/Users/malik/Desktop/Malik/University/General - GP 1  2023 A/G/Data/img/20230101002212_000025.JPG')

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (640, 480))

# Initialize CSV writer
csv_file = open('output.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['frame_nmr', 'car_id', 'car_bbox', 'license_bbox', 'license_text', 'license_bbox_score', 'license_text_score'])

# Main loop
vehicles = [2, 3, 5, 7]
frame_nmr = -1
ret = True

while ret:
    frame_nmr += 1
    ret, frame = cap.read()
    if ret:
        results[frame_nmr] = {}
        detections = coco_model(frame)[0]
        detections_ = [[x1, y1, x2, y2, score] for x1, y1, x2, y2, score, class_id in detections.boxes.data.tolist() if int(class_id) in vehicles]
        track_ids = mot_tracker.update(np.array(detections_))
        license_plates = license_plate_detector(frame)[0]
        
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)
            if car_id != -1:
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]
                license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop)
                
                if license_plate_text is not None:
                    results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                                  'license_plate': {'bbox': [x1, y1, x2, y2],
                                                                    'text': license_plate_text,
                                                                    'bbox_score': score,
                                                                    'text_score': license_plate_text_score}}
                    
                    # Write to CSV
                    csv_writer.writerow([frame_nmr, car_id, [xcar1, ycar1, xcar2, ycar2], [x1, y1, x2, y2], license_plate_text, score, license_plate_text_score])
                    
                    # Draw bounding boxes and text on the frame
                    cv2.rectangle(frame, (int(xcar1), int(ycar1)), (int(xcar2), int(ycar2)), (0, 255, 0), 2)
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
                    cv2.putText(frame, license_plate_text, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        # Write the frame to the output video
        out.write(frame)

# Release video and close CSV
cap.release()
out.release()
csv_file.close()



0: 384x640 9 cars, 1 truck, 245.1ms
Speed: 5.3ms preprocess, 245.1ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 501.6ms
Speed: 2.6ms preprocess, 501.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)


[([[0, 1], [121, 1], [121, 37], [0, 37]], '4523lusR', 0.40241477911945195)]
[]
